In [6]:
import seedir as sd
sd.seedir("../src", style='lines', itemlimit=10, depthlimit=2, exclude_folders="__pycache__")

src/
├─plotting_functions.py
├─config.json
├─model_training.py
├─data_loader.py
├─utils/
│ ├─config.py
│ └─args.py
├─model.py
├─trainer.py
└─main.py
